In [1]:
from myfunct import *
import pandas as pd
import numpy as np
import seaborn as sns

import gensim
import spacy 

In [2]:
import gensim.corpora as corpora

In [22]:
from IPython.display import IFrame
IFrame('../assets/simplex.html', 1000, 1000)

/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df = pd.read_csv('../data/homeless_posts.csv')
token_text, corpus, id2word =  load_data('topic_1.01')

In [5]:
def lemmatization(texts, allowed_postages=['NOUN','ADJ','VERB','ADV']):
    
    '''
    allowed_postage : the parts of speach we want to keep [DEFAULT: 'NOUN','ADJ','VERB','ADV'] 
    '''
    
          # load in spacy sm web model 
    nlp = spacy.load('en_core_web_sm', diasble = ['parser','ner']) # computaltionally expensize aspects 
    texts_out = [] # output
    
    # for each post in the corpus
    # iterate over texts
    for text in texts:  
        # creates spacy doc object containing vectorized contextual information like Parts of Speech (pos) 
        doc = nlp(text)
        

        ## The below code was used in experimenatation with stopwords, 
        ## but our topic filter was not improved by our applications of this feature
        ## code has been left for posterity.
        
            ## adding words to spacy pipeline stopword list 

      

        # list for holding lemmatized tokens
        new_text = []
        # iterate over each token
        for token in doc:
            # only keep the desired pos
            if token.pos_ in allowed_postages:
                if not token.is_stop == True:
                    # reducing model complexity by reducing tokens to lemma_ 
                    new_text.append(token.lemma_)   
                    # print(token.lemma_)

        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [6]:
#my_stop_words = ['can','will','so','now','see','white','brown','go','well','need','help','want']
  

In [7]:
def gen_words(texts):
    
    nlp = spacy.load('en_core_web_sm', diasble = ['parser','ner']) # computaltionally expensize aspects 

    
    my_stop_words = ['homeless','homelessness','animal','dog','thank','want','clothing','goodwill','need','neighborhood','community','order','offer']
    final = [] 
    for text in texts:
        # new = mwe.tokenize(text)
        # Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.
        new = gensim.utils.simple_preprocess(text , deacc=True)# – Remove accent marks from tokens using 
        new = [ w for w in new if not w in my_stop_words ]
        final.append(new)
        
    return(final)

In [8]:
from gensim.models import CoherenceModel 

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b, c ):
    
    """
    corpus: text body in string form
    dictionary: id2word 
    k: num_topics
    a: alpha - document topic density
    b: beta- word topic density 
    
    """
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=c,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [9]:
lemmatized_posts = lemmatization(df['post'])

In [10]:
data_words = gen_words(lemmatized_posts)

In [11]:
# id to word dictionary 
id2word = corpora.Dictionary(data_words)

corpus = []

for text in data_words:
    # create a bag of word vectors from our texts word id# and frequency 
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]


In [12]:
# Topics range
min_topics = 3
max_topics = 16
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# # Chunk Size
chunks = list(np.linspace(20 , 100 , 5))

# Alpha parameter
alpha = list(np.arange(0.15, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# (0.01, 1, 0.3)
# Beta parameter
beta = list(np.arange(0.15, 1, 0.3))
beta.append('symmetric')

# result df entry format
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Chunks': [],
                 'Coherence': []
                }# Can take a long time to run

In [13]:
# iterate through number of topics
for k in topics_range:
    # iterate through alpha values
    for a in alpha:
          # iterare through beta values
        for b in beta:
            # iterate through chunksizes
            for c in chunks:
                 # get cohenerence value

                cv = compute_coherence_values( corpus, id2word, 
                                              k, a, b, c = c)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Chunks'].append(c)
                model_results['Coherence'].append(cv)

In [14]:
df = pd.DataFrame(model_results)
# create a chart grouped by parameters
chart = df.groupby(['Topics','Alpha','Beta','Chunks']).describe()
# set color of gradient
cm = sns.light_palette("red", as_cmap=True);
chart = chart.style.background_gradient(cmap=cm);
chart = chart.highlight_max(color = 'grey');

In [15]:
chart

/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/pandas/io/formats/style.py:1264: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/pandas/io/formats/style.py:1265: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax
/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/pandas/io/formats/style.py:1547: RuntimeWarning: All-NaN slice encountered
  extrema = data == np.nanmax(data.to_numpy())


In [16]:
view = df[df.Topics < 7]
view[view.Coherence > view.Coherence.max() - .05]

,Topics,Alpha,Beta,Chunks,Coherence
26,3,0.45,0.45,40.0,0.488881
31,3,0.45,0.75,40.0,0.508072
51,3,0.75,0.75,40.0,0.503646
111,4,0.15,0.75,40.0,0.487488
113,4,0.15,0.75,80.0,0.496225
131,4,0.45,0.75,40.0,0.537256
171,4,symmetric,0.75,40.0,0.496661
186,4,asymmetric,0.45,40.0,0.500844
211,5,0.15,0.75,40.0,0.520548
250,5,0.75,0.75,20.0,0.502665


In [17]:
# LDA visualizations
import pyLDAvis
import pyLDAvis.gensim_models

from IPython.display import HTML
css_str = '<style> \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str))

In [18]:
# saved Model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           chunksize=40,
                                           passes=100,
                                           alpha= 0.15,
                                           eta=0.25,
                                           #per_word_topics = True
                                           )

pyLDAvis.enable_notebook() # lets us see visualizations in jupyter 
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R= 10) # model / corpus / id2word dict / 
vis

/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.229086 -0.258743       1        1  40.381109
2     -0.140606 -0.250488       2        1  19.711465
7      0.144036  0.212983       3        1   9.853104
3     -0.099572  0.161635       4        1   7.115290
5     -0.183543  0.036456       5        1   5.374476
8     -0.093294 -0.031390       6        1   4.283506
0      0.038846 -0.098418       7        1   4.124731
11     0.114644  0.024208       8        1   2.897224
10     0.034308  0.020272       9        1   1.507271
4      0.012721  0.082457      10        1   1.414492
6     -0.030884  0.028205      11        1   1.185246
1     -0.000875  0.029471      12        1   1.023150
9     -0.013703  0.027431      13        1   0.587298
12    -0.011163  0.015922      14        1   0.541640, topic_info=           Term        Freq       Total Category  logprob  loglift
73         food   80.000000   80.000000  Default  10.0000  10.0000
60         help  302.000000  302.000000  Default   9.0000   9.0000
71       donate  110.000000  110.000000  Default   8.0000   8.0000
87         city  109.000000  109.000000  Default   7.0000   7.0000
666        item   61.000000   61.000000  Default   6.0000   6.0000
...         ...         ...         ...      ...      ...      ...
3749  worthless    0.230366    1.885695  Topic14  -6.3235   3.1159
3434    dentist    0.197808    1.824978  Topic14  -6.4758   2.9963
3435        tmj    0.197808    1.824978  Topic14  -6.4758   2.9963
3183     market    0.328627    3.551642  Topic14  -5.9682   2.8381
726       worth    0.251347    2.865030  Topic14  -6.2363   2.7848

[211 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
648       4  0.144651  accept
648       8  0.723255  accept
2858      6  0.333394  admire
2858     11  0.333394  admire
1902      9  0.305881  advise
...     ...       ...     ...
726       5  0.349036   worth
502       6  0.924206   youth
1199      2  0.099033    zone
1199      4  0.841777    zone
51        4  0.914796  zoning

[258 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 3, 8, 4, 6, 9, 1, 12, 11, 5, 7, 2, 10, 13])

In [23]:
pyLDAvis.save_html(vis,'../assets/TOPICS_FINAL.exp.html')

/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
lda_model.save("../data/models/topic_14.fin.model")

/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
